<a href="https://colab.research.google.com/github/paraery/parsingResume/blob/main/spacyWithDistillBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
def get_tokens_with_entities(raw_text: str):
    # split the text by spaces only if the space does not occur between square brackets
    # we do not want to split "multi-word" entity value yet
    
    raw_tokens = re.split(r"\s(?![^\[]*\])", raw_text)

    # a regex for matching the annotation according to our notation [entity_value](entity_name)
    entity_value_pattern = r"\[(?P<value>.+?)\]\((?P<entity>.+?)\)"
    entity_value_pattern_compiled = re.compile(entity_value_pattern, flags=re.I|re.M)

    tokens_with_entities = []
    lstEnt=[]
    txt = raw_text
    for raw_token in raw_tokens:
        match = entity_value_pattern_compiled.match(raw_token)
        if match:
            raw_entity_name, raw_entity_value = match.group("entity"), match.group("value")
            
            # we prefix the name of entity differently
            # B- indicates beginning of an entity
            # I- indicates the token is not a new entity itself but rather a part of existing one
            #txt = raw_text[raw_text.find(str(raw_entity_value))-1 : ]
            txt = raw_text.replace(raw_text[int(raw_text.find(raw_entity_value))-1],'',1)
            txt = txt.replace('](PERSON)','')
            #print(txt)
            lstdata = ()
            entity_name = f"{raw_entity_name}"
            entName = createSubEntity(txt.find(raw_entity_value), txt.find(raw_entity_value) + len(raw_entity_value), entity_name)
            lstEnt.append(entName)
                #tokens_with_entities.append((raw_entity_token, entity_name))
    d={"entities": lstEnt}
            #print(lstEnt)
    lstdata=(txt,d)
    tokens_with_entities.append(lstdata)
    return lstdata
def createSubEntity(start,end,tag):
    s = (int(start),int(end),tag)
    return s

In [17]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/textfortraining8.csv')
lstTxt = df['text'].values.tolist()

In [18]:
#train_data=[]
for a in lstTxt:
    train_data.append(get_tokens_with_entities(a))

In [19]:
train=[]
train=train_data

In [20]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/testData.csv')
lstTest = df['text'].values.tolist()

In [21]:
test=[]
for a in lstTest:
    test.append(get_tokens_with_entities(a))

In [22]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.0 which is incompatible.


In [23]:
!pip install spacy_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.2 MB/s eta 0:00:00


In [24]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [25]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [26]:
!python -m spacy init fill-config /content/drive/MyDrive/Spacy/base_config.cfg /content/drive/MyDrive/Spacy/config.cfg

2023-03-04 11:47:03.344387: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 11:47:03.344480: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 11:47:03.344497: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/Spacy/config.cfg
You can now add your data and train your pipeline:
python -m spacy train co

In [27]:
def get_spacy_doc(file,data):
  nlp = spacy.blank("en")
  db = DocBin()
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']
    ents = []
    entity_indices = []
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity==True:
        continue
      entity_indices = entity_indices+list(range(start,end))

      try:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
      except:
        continue
      if span is None:
        err_data = str([start,end])+" "+str(text)+"\n"
        file.write(err_data)
      else:
        ents.append(span)
      
      doc.ents = ents
      db.add(doc)
  return db

In [28]:
file = open('drive/MyDrive/Spacy/error.txt','w')
db = get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 87/87 [00:00<00:00, 164.68it/s]


In [29]:
!python -m spacy train /content/drive/MyDrive/Spacy/config.cfg --output ./outputSpacyDistillBert --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-03-04 11:49:42.823756: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 11:49:42.823859: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-04 11:49:42.823878: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
✔ Created output directory: outputSpacyDistillBert
ℹ Saving to output directory: outputSpacyDistillBert
ℹ Using GPU: 0

=========================== Initializing pipeli

In [30]:
nlp = spacy.load('outputSpacyDistillBert/model-best')

In [31]:
import pandas as pd
df = pd.read_csv('drive/MyDrive/textInResume.csv')
dataTest = df['text'].values.tolist()

In [32]:
i=0
for d in dataTest:
  doc = nlp(d)
  print("## doc ",i," ##")
  for ent in doc.ents:
    print(ent.text,"----",ent.label_)
  i = i+1
  print("   ")

## doc  0  ##
 certification ---- PERSON
 automation ---- PERSON
 proficient ---- PERSON
 erp ---- PERSON
supavit attagomol ---- PERSON
   
## doc  1  ##
nisaratwisesbantao bachelor ---- PERSON
   
## doc  2  ##
thirawit jirarungroj ---- PERSON
   
## doc  3  ##
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERSON
 ( ---- PERS

Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors


## doc  9  ##
nattacha kingbualuang ---- PERSON
   
## doc  10  ##
purawat ruangsri ---- PERSON
   
## doc  11  ##
nutratanon mahakhet ---- PERSON
   
## doc  12  ##
kitisak thossaensin ---- PERSON
   
## doc  13  ##
suebtas limsaihua ---- PERSON
   
## doc  14  ##
navara sirijarusvong ---- PERSON
   
## doc  15  ##
chayanit sripradit ---- PERSON
   
## doc  16  ##
pimnares puto ---- PERSON
   
## doc  17  ##
saran thitawiriyayos ---- PERSON
   
## doc  18  ##
pakpoom poodsud ---- PERSON
   
## doc  19  ##
kanittha setthapitayakul ---- PERSON
   
## doc  20  ##
   
## doc  21  ##
pawith panyasirikul ---- PERSON
   
## doc  22  ##
chotitouch supalanunt ---- PERSON
   
## doc  23  ##
aussadach masun ---- PERSON
   
## doc  24  ##
teeratach jearapaganon ---- PERSON
̄© ---- PERSON
̄ ̄£ ---- PERSON
   
## doc  25  ##
pratan srikamonpattanawut ---- PERSON
   
## doc  26  ##
natthapat phusrisom ---- PERSON
   
## doc  27  ##
chayanon juntarapartsavorn ---- PERSON
   
## doc  28  ##
narurong s